In [30]:
import numpy as np
import pandas as pd
from sklearn.utils import shuffle
from tensorflow import set_random_seed

#keras package
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.models import Model, load_model
from keras.preprocessing import image
from keras.utils.vis_utils import model_to_dot
from keras.initializers import glorot_uniform
from keras.wrappers.scikit_learn import KerasClassifier,KerasRegressor
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.grid_search import GridSearchCV 
from keras import regularizers
from random import choice
import pydot
import graphviz
from IPython.display import SVG
import keras.backend as K
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
np.random.seed(42)


## 1.读取-划分数据

In [2]:
def importdata():
    path = r"C:\Users\hasee\workspace\workspace\tomo\data2.txt"
    df = pd.read_table(path)
    columns = df.columns
    #print(columns)

    dfdata = df[['latgpsutm', 'lnggpsutm', 'lnglacutm', 'latlacutm']]
    #print(len(dfdata))
    dfdata = dfdata.drop_duplicates()#删去重复项
    #print(len(dfdata))
    return dfdata

df = importdata()

随机70%训练集切分-方法1：

In [3]:
df1 = shuffle(df)
df1 = df.set_index([list(range(len(df1)))])
index = int(len(df1)*0.7)
df1_train=df1[:index]
df1_test=df1[index:]

随机70%训练集切分-方法2：

In [4]:
df_train=df.sample(frac=0.7)
df_train=df_train.reset_index(drop=True)

In [5]:
df_train.isnull().values.any()

False

In [6]:
list(df_train)

['latgpsutm', 'lnggpsutm', 'lnglacutm', 'latlacutm']

In [51]:
X_train=df1[['lnglacutm', 'latlacutm']]
y_train=df1[['latgpsutm', 'lnggpsutm']]

In [79]:
val_1=df1.loc[10:20]
val_1

,latgpsutm,lnggpsutm,lnglacutm,latlacutm
10,3.462141e+06,357312.1664,356965.6919,3.462668e+06
11,3.462141e+06,357312.1664,357405.7554,3.461803e+06
12,3.462425e+06,357110.5482,357405.7554,3.461803e+06
13,3.462212e+06,357045.4753,356788.6773,3.462205e+06
14,3.462329e+06,356804.1912,356915.2664,3.462539e+06
15,3.462329e+06,356804.1912,356698.2993,3.462768e+06
16,3.462617e+06,356729.9513,356698.2993,3.462768e+06
17,3.462329e+06,356804.1912,356608.2450,3.462769e+06
18,3.462617e+06,356729.9513,356915.2664,3.462539e+06
19,3.462425e+06,357110.5482,356965.6919,3.462668e+06


# Build the model

In [9]:
K.clear_session()
def NN_model(optimizer='adam',hidden_units=100,dropout_rate=0.3,L2_rate=.001):
    model=Sequential()#设置为Sequential模型
    #添加第一层hidden layer
    model.add(Dense(hidden_units,activation='relu',input_dim=2,kernel_regularizer=regularizers.l2(L2_rate)))
    #添加dropout（正则化，减少overfit）
    model.add(Dropout(dropout_rate))
    #添加输出层
    model.add(Dense(2,activation='relu',kernel_regularizer=regularizers.l2(L2_rate)))
    #编译
    model.compile(loss='mean_squared_error',optimizer=optimizer)
    return model

In [10]:
#kfold=StratifiedKFold(n_splits=5,random_state=42)
#nn_reg=KerasRegressor(build_fn=NN_model,nb_epoch=40,batch_size=50,verbose=-1)
#nn_reg.fit(X_train,y_train)
nn_model=NN_model()

In [39]:
nn_model.fit(X_train.as_matrix(),y_train.as_matrix(),epochs=100,validation_split=0.2,batch_size=50,verbose=-1)#callbacks='early_stop')

Train on 893 samples, validate on 224 samples
Epoch 1/100
Epoch 2/100
Epoch 3/100
Epoch 4/100
Epoch 5/100
Epoch 6/100
Epoch 7/100
Epoch 8/100
Epoch 9/100
Epoch 10/100
Epoch 11/100
Epoch 12/100
Epoch 13/100
Epoch 14/100
Epoch 15/100
Epoch 16/100
Epoch 17/100
Epoch 18/100
Epoch 19/100
Epoch 20/100
Epoch 21/100
Epoch 22/100
Epoch 23/100
Epoch 24/100
Epoch 25/100
Epoch 26/100
Epoch 27/100
Epoch 28/100
Epoch 29/100
Epoch 30/100
Epoch 31/100
Epoch 32/100
Epoch 33/100
Epoch 34/100
Epoch 35/100
Epoch 36/100
Epoch 37/100
Epoch 38/100
Epoch 39/100
Epoch 40/100
Epoch 41/100
Epoch 42/100
Epoch 43/100
Epoch 44/100
Epoch 45/100
Epoch 46/100
Epoch 47/100
Epoch 48/100
Epoch 49/100
Epoch 50/100
Epoch 51/100
Epoch 52/100
Epoch 53/100
Epoch 54/100
Epoch 55/100
Epoch 56/100
Epoch 57/100
Epoch 58/100
Epoch 59/100
Epoch 60/100
Epoch 61/100
Epoch 62/100
Epoch 63/100
Epoch 64/100
Epoch 65/100
Epoch 66/100
Epoch 67/100
Epoch 68/100
Epoch 69/100
Epoch 70/100
Epoch 71/100
Epoch 72/100
Epoch 73/100
Epoch 74/100
E

In [42]:
nn_model.evaluate(X_test.as_matrix(),y_test.as_matrix())

479/479 [==============================] - ETA: 0s


3056360622.764092

In [14]:
test_X=df1_test[['lnglacutm', 'latlacutm']].iloc[0:2].as_matrix()
test_y=df1_test[['latgpsutm', 'lnggpsutm']].iloc[0:2].as_matrix()

In [16]:
nn_model.predict(test_X)

array([[ 3448758.  ,   357042.25],
       [ 3448758.  ,   357042.25]], dtype=float32)

In [17]:
test_y

array([[ 3462168.7389,   357200.1846],
       [ 3462169.8034,   357217.0202]])

# 2. added the call backs

In [28]:
filepath="./final_model.ckpt"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=-1, save_best_only=True, mode='min')

earlystop= EarlyStopping(monitor='val_loss', patience=10, mode='min') 
nn_model.fit(X_train,y_train,epochs=100,validation_split=0.2,\
             batch_size=50,verbose=-1,callbacks=[earlystop,checkpoint])

Train on 893 samples, validate on 224 samples
Epoch 1/100
Epoch 2/100
Epoch 3/100
Epoch 4/100
Epoch 5/100
Epoch 6/100
Epoch 7/100
Epoch 8/100
Epoch 9/100
Epoch 10/100
Epoch 11/100
Epoch 12/100
Epoch 13/100
Epoch 14/100
Epoch 15/100
Epoch 16/100
Epoch 17/100
Epoch 18/100
Epoch 19/100
Epoch 20/100
Epoch 21/100


# 3. Cross_Validation tuning the parameter

In [88]:
MAX_EPOCH=100#最大epoch，实际应用early_stop
n_iteration=20#迭代20次
best_score=0#初始化最佳loss分数
#设置验证集（cv=5）
val_set_1=df1.loc[:318]
val_set_2=df1.loc[319:2*319-1]
val_set_3=df1.loc[2*319:3*319-1]
val_set_4=df1.loc[3*319:4*319-1]
val_set_5=df1.loc[4*319:]
for iteration in range(n_iteration):
    loss_set=[]
    fold_num=0
    #set_random_seed(iteration)#设置随机种子
    np.random.seed(iteration) 
        #设置随机搜索参数
    param_random={
        'hidden_units':choice(range(50,150,1)),#随机选择其中一个
        'dropout_rate':choice([.2,.22,.24,.26,.28,.30,.32,.34]),
        'optimizer':choice([ 'Adagrad', 'Adadelta', 'Adam', 'Adamax']),
        'L2':choice([.001,.002,.003,.004])
    }
    print("%d iteration has the parameter :"%(iteration))
    print(param_random)
    #kfold=StratifiedKFold(n_splits=5,random_state=42,shuffle=True)
    #for i ,(train_index,val_index) in enumerate(kfold.split(X_train,y_train)):
    for val_set in [val_set_1,val_set_2,val_set_3,val_set_4,val_set_5]:
        fold_num+=1
        train_x=df1.drop(val_set.index)[['lnglacutm', 'latlacutm']]
        train_y=df1.drop(val_set.index)[['latgpsutm', 'lnggpsutm']]
        val_x,val_y=val_set[['lnglacutm', 'latlacutm']],val_set[['latgpsutm', 'lnggpsutm']]
        print("Fold ",fold_num," Start")
        #带入参数
        nn_model=NN_model(hidden_units=param_random['hidden_units'],\
                          optimizer=param_random['optimizer'],\
                          dropout_rate=param_random['dropout_rate'],\
                          L2_rate=param_random['L2']
                         )
        #输出模型
        filepath="./model_"+str(iteration)+"_and_"+str(fold_num)+".ckpt"
        checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=-1, save_best_only=True, mode='min')
        #设置early_stop
        earlystop= EarlyStopping(monitor='val_loss', patience=10, mode='min') 
        nn_model.fit(train_x.as_matrix(),train_y.as_matrix(),epochs=MAX_EPOCH,validation_split=0.2,\
                  batch_size=50,verbose=-1,callbacks=[earlystop,checkpoint])
        loss=nn_model.evaluate(val_x.as_matrix(),val_y.as_matrix())
        loss_set.append(loss)
        print("%dFold's loss value is %f"%(fold_num,loss))
    #平均5个fold的loss值，得到这个参数下得loss分数
    loss_score=np.mean(loss_set)
    print("%d iteration's loss value is %f"%(iteration,loss_score))
    if iteration == 0:
        best_score=loss_score
    #如果loss分数小于最好分数，这个loss分数是新的最佳loss分数
    if loss_score<best_score:
        best_score=loss_score
        print("%d iteration has best score %f"%(iteration,best_score))

    
        
        
                

0 iteration has the parameter :
{'hidden_units': 128, 'dropout_rate': 0.34, 'optimizer': 'Adam', 'L2': 0.001}
Fold  1  Start
Train on 1021 samples, validate on 256 samples
Epoch 1/100
Epoch 2/100
Epoch 3/100
Epoch 4/100
Epoch 5/100
Epoch 6/100
Epoch 7/100
Epoch 8/100
Epoch 9/100
Epoch 10/100
Epoch 11/100
Epoch 12/100
Epoch 13/100
Epoch 14/100
Epoch 15/100
Epoch 16/100
Epoch 17/100
Epoch 18/100
Epoch 19/100
Epoch 20/100
Epoch 21/100
Epoch 22/100
Epoch 23/100
Epoch 24/100
319/319 [==============================] - ETA:  - ETA: 0s
1Fold's loss value is 894845152.702194
Fold  2  Start
Train on 1021 samples, validate on 256 samples
Epoch 1/100
Epoch 2/100
Epoch 3/100
Epoch 4/100
Epoch 5/100
Epoch 6/100
Epoch 7/100
Epoch 8/100
Epoch 9/100
Epoch 10/100
Epoch 11/100
Epoch 12/100
Epoch 13/100
Epoch 14/100
Epoch 15/100
Epoch 16/100
Epoch 17/100
Epoch 18/100
Epoch 19/100
Epoch 20/100
Epoch 21/100
Epoch 22/100
Epoch 23/100
Epoch 24/100
Epoch 25/100
Epoch 26/100
Epoch 27/100
Epoch 28/100
Epoch 29/1

Epoch 18/100
Epoch 19/100
Epoch 20/100
Epoch 21/100
Epoch 22/100
Epoch 23/100
Epoch 24/100
Epoch 25/100
Epoch 26/100
Epoch 27/100
Epoch 28/100
Epoch 29/100
Epoch 30/100
Epoch 31/100
Epoch 32/100
319/319 [==============================] - ETA: 0s
4Fold's loss value is 6452489591.573668
Fold  5  Start
Train on 1020 samples, validate on 256 samples
Epoch 1/100
Epoch 2/100
Epoch 3/100
Epoch 4/100
Epoch 5/100
Epoch 6/100
Epoch 7/100
Epoch 8/100
Epoch 9/100
Epoch 10/100
Epoch 11/100
Epoch 12/100
Epoch 13/100
Epoch 14/100
Epoch 15/100
Epoch 16/100
Epoch 17/100
Epoch 18/100
Epoch 19/100
Epoch 20/100
Epoch 21/100
Epoch 22/100
Epoch 23/100
Epoch 24/100
Epoch 25/100
Epoch 26/100
Epoch 27/100
Epoch 28/100
Epoch 29/100
Epoch 30/100
Epoch 31/100
Epoch 32/100
Epoch 33/100
Epoch 34/100
Epoch 35/100
Epoch 36/100
Epoch 37/100
Epoch 38/100
Epoch 39/100
Epoch 40/100
Epoch 41/100
Epoch 42/100
Epoch 43/100
Epoch 44/100
Epoch 45/100
320/320 [==============================] - ETA: 0s
5Fold's loss value is 477

Epoch 11/100
Epoch 12/100
Epoch 13/100
Epoch 14/100
Epoch 15/100
Epoch 16/100
Epoch 17/100
Epoch 18/100
Epoch 19/100
Epoch 20/100
Epoch 21/100
Epoch 22/100
Epoch 23/100
Epoch 24/100
Epoch 25/100
Epoch 26/100
Epoch 27/100
Epoch 28/100
Epoch 29/100
Epoch 30/100
Epoch 31/100
Epoch 32/100
Epoch 33/100
319/319 [==============================] - ETA: 0s
2Fold's loss value is 73840993.053292
Fold  3  Start
Train on 1021 samples, validate on 256 samples
Epoch 1/100
Epoch 2/100
Epoch 3/100
Epoch 4/100
Epoch 5/100
Epoch 6/100
Epoch 7/100
Epoch 8/100
Epoch 9/100
Epoch 10/100
Epoch 11/100
Epoch 12/100
Epoch 13/100
Epoch 14/100
Epoch 15/100
Epoch 16/100
Epoch 17/100
Epoch 18/100
Epoch 19/100
Epoch 20/100
Epoch 21/100
Epoch 22/100
Epoch 23/100
Epoch 24/100
Epoch 25/100
Epoch 26/100
Epoch 27/100
Epoch 28/100
Epoch 29/100
Epoch 30/100
Epoch 31/100
Epoch 32/100
Epoch 33/100
Epoch 34/100
Epoch 35/100
Epoch 36/100
Epoch 37/100
Epoch 38/100
Epoch 39/100
Epoch 40/100
Epoch 41/100
Epoch 42/100
Epoch 43/100


Epoch 3/100
Epoch 4/100
Epoch 5/100
Epoch 6/100
Epoch 7/100
Epoch 8/100
Epoch 9/100
Epoch 10/100
Epoch 11/100
Epoch 12/100
Epoch 13/100
Epoch 14/100
Epoch 15/100
Epoch 16/100
Epoch 17/100
Epoch 18/100
Epoch 19/100
Epoch 20/100
Epoch 21/100
Epoch 22/100
Epoch 23/100
Epoch 24/100
Epoch 25/100
Epoch 26/100
Epoch 27/100
Epoch 28/100
Epoch 29/100
Epoch 30/100
Epoch 31/100
Epoch 32/100
319/319 [==============================] - ETA: 0s
3Fold's loss value is 4298506280.125392
Fold  4  Start
Train on 1021 samples, validate on 256 samples
Epoch 1/100
Epoch 2/100
Epoch 3/100
Epoch 4/100
Epoch 5/100
Epoch 6/100
Epoch 7/100
Epoch 8/100
Epoch 9/100
Epoch 10/100
Epoch 11/100
Epoch 12/100
Epoch 13/100
Epoch 14/100
Epoch 15/100
Epoch 16/100
Epoch 17/100
Epoch 18/100
Epoch 19/100
Epoch 20/100
Epoch 21/100
Epoch 22/100
Epoch 23/100
Epoch 24/100
Epoch 25/100
Epoch 26/100
Epoch 27/100
Epoch 28/100
Epoch 29/100
Epoch 30/100
Epoch 31/100
Epoch 32/100
Epoch 33/100
Epoch 34/100
Epoch 35/100
Epoch 36/100
Epoch

Epoch 23/100
Epoch 24/100
Epoch 25/100
319/319 [==============================] - ETA: 0s
1Fold's loss value is 1490048957.793103
Fold  2  Start
Train on 1021 samples, validate on 256 samples
Epoch 1/100
Epoch 2/100
Epoch 3/100
Epoch 4/100
Epoch 5/100
Epoch 6/100
Epoch 7/100
Epoch 8/100
Epoch 9/100
Epoch 10/100
Epoch 11/100
Epoch 12/100
Epoch 13/100
Epoch 14/100
Epoch 15/100
Epoch 16/100
Epoch 17/100
Epoch 18/100
Epoch 19/100
Epoch 20/100
Epoch 21/100
Epoch 22/100
Epoch 23/100
Epoch 24/100
Epoch 25/100
Epoch 26/100
Epoch 27/100
Epoch 28/100
Epoch 29/100
Epoch 30/100
Epoch 31/100
Epoch 32/100
Epoch 33/100
Epoch 34/100
Epoch 35/100
Epoch 36/100
Epoch 37/100
Epoch 38/100
Epoch 39/100
Epoch 40/100
Epoch 41/100
Epoch 42/100
Epoch 43/100
Epoch 44/100
Epoch 45/100
Epoch 46/100
Epoch 47/100
Epoch 48/100
Epoch 49/100
Epoch 50/100
Epoch 51/100
Epoch 52/100
Epoch 53/100
319/319 [==============================] - ETA: 0s
2Fold's loss value is 413254682.181818
Fold  3  Start
Train on 1021 samples, 

Epoch 2/100
Epoch 3/100
Epoch 4/100
Epoch 5/100
Epoch 6/100
Epoch 7/100
Epoch 8/100
Epoch 9/100
Epoch 10/100
Epoch 11/100
Epoch 12/100
Epoch 13/100
Epoch 14/100
Epoch 15/100
Epoch 16/100
Epoch 17/100
Epoch 18/100
Epoch 19/100
Epoch 20/100
Epoch 21/100
Epoch 22/100
Epoch 23/100
319/319 [==============================] - ETA: 0s
2Fold's loss value is 2821227479.874608
Fold  3  Start
Train on 1021 samples, validate on 256 samples
Epoch 1/100
Epoch 2/100
Epoch 3/100
Epoch 4/100
Epoch 5/100
Epoch 6/100
Epoch 7/100
Epoch 8/100
Epoch 9/100
Epoch 10/100
Epoch 11/100
Epoch 12/100
Epoch 13/100
Epoch 14/100
Epoch 15/100
Epoch 16/100
Epoch 17/100
Epoch 18/100
Epoch 19/100
Epoch 20/100
Epoch 21/100
Epoch 22/100
Epoch 23/100
Epoch 24/100
Epoch 25/100
Epoch 26/100
Epoch 27/100
Epoch 28/100
Epoch 29/100
Epoch 30/100
Epoch 31/100
Epoch 32/100
Epoch 33/100
Epoch 34/100
Epoch 35/100
Epoch 36/100
Epoch 37/100
Epoch 38/100
Epoch 39/100
Epoch 40/100
Epoch 41/100
Epoch 42/100
Epoch 43/100
Epoch 44/100
Epoch 

Epoch 14/100
Epoch 15/100
Epoch 16/100
Epoch 17/100
Epoch 18/100
Epoch 19/100
Epoch 20/100
Epoch 21/100
Epoch 22/100
319/319 [==============================] - ETA: 0s
4Fold's loss value is 168478296.827586
Fold  5  Start
Train on 1020 samples, validate on 256 samples
Epoch 1/100
Epoch 2/100
Epoch 3/100
Epoch 4/100
Epoch 5/100
Epoch 6/100
Epoch 7/100
Epoch 8/100
Epoch 9/100
Epoch 10/100
Epoch 11/100
Epoch 12/100
Epoch 13/100
Epoch 14/100
Epoch 15/100
Epoch 16/100
Epoch 17/100
Epoch 18/100
Epoch 19/100
Epoch 20/100
Epoch 21/100
Epoch 22/100
Epoch 23/100
Epoch 24/100
320/320 [==============================] - ETA: 0s
5Fold's loss value is 792370284.800000
15 iteration's loss value is 288357913.576928
15 iteration has best score 288357913.576928
16 iteration has the parameter :
{'hidden_units': 88, 'dropout_rate': 0.34, 'optimizer': 'Adam', 'L2': 0.002}
Fold  1  Start
Train on 1021 samples, validate on 256 samples
Epoch 1/100
Epoch 2/100
Epoch 3/100
Epoch 4/100
Epoch 5/100
Epoch 6/100
Epo

319/319 [==============================] - ETA: 0s
2Fold's loss value is 3546886139.184953
Fold  3  Start
Train on 1021 samples, validate on 256 samples
Epoch 1/100
Epoch 2/100
Epoch 3/100
Epoch 4/100
Epoch 5/100
Epoch 6/100
Epoch 7/100
Epoch 8/100
Epoch 9/100
Epoch 10/100
Epoch 11/100
Epoch 12/100
Epoch 13/100
Epoch 14/100
Epoch 15/100
Epoch 16/100
Epoch 17/100
Epoch 18/100
Epoch 19/100
Epoch 20/100
Epoch 21/100
Epoch 22/100
Epoch 23/100
Epoch 24/100
Epoch 25/100
Epoch 26/100
Epoch 27/100
Epoch 28/100
Epoch 29/100
Epoch 30/100
Epoch 31/100
319/319 [==============================] - ETA: 0s
3Fold's loss value is 5981776904192.000000
Fold  4  Start
Train on 1021 samples, validate on 256 samples
Epoch 1/100
Epoch 2/100
Epoch 3/100
Epoch 4/100
Epoch 5/100
Epoch 6/100
Epoch 7/100
Epoch 8/100
Epoch 9/100
Epoch 10/100
Epoch 11/100
Epoch 12/100
Epoch 13/100
Epoch 14/100
Epoch 15/100
Epoch 16/100
Epoch 17/100
Epoch 18/100
Epoch 19/100
Epoch 20/100
Epoch 21/100
Epoch 22/100
Epoch 23/100
Epoch 2

45